In [1]:
import torch
from torch_geometric.data import Data
import pandas as pd
import pickle
import numpy as np

from torch_geometric.nn import SAGEConv
import torch
import torch_geometric
import torch.nn.functional as F
torch_geometric.set_debug(True)

# Helper Functions

In [3]:
def read_data(nodes_df_path, edges_df_path, subject_mapping_path):
    nodes_df = pd.read_csv(nodes_df_path)
    edges_df = pd.read_csv(edges_df_path)
    with open(subject_mapping_path, 'rb') as f:
        subject_mapping = pickle.load(f)
    return nodes_df, edges_df, subject_mapping


def get_node_id_mapping(nodes_df):
    node_id_mapping, inverse_node_id_mapping = dict(), dict()
    for i, node_id in enumerate(nodes_df['nodeId']):
        node_id_mapping[i] = node_id
        inverse_node_id_mapping[node_id] = i
    return node_id_mapping, inverse_node_id_mapping

In [ ]:
def get_feature_vectors():
    # TODO: Complete this function
    pass


def get_edges():
    # TODO: Complete this function
    pass


def get_labels():
    # TODO: Complete this function
    pass

# Graph Generation

In [4]:
nodes_df_path = 'nodes.csv'
edges_df_path = 'edges.csv'
subject_mapping_path = 'subject_mapping.pkl'
nodes_df, edges_df, subject_mapping = read_data(nodes_df_path, edges_df_path, subject_mapping_path)

In [ ]:
node_id_mapping, inverse_node_id_mapping = get_node_id_mapping(nodes_df)
# TODO: These functions need to be implemented. You can decide what are the input arguments to these functions.
x = get_feature_vectors()
edge_index = get_edges()
y = get_labels()

In [ ]:
with open('indices_dict_part2.pkl', 'rb') as f:
    indices_dict = pickle.load(f)

In [ ]:
train_mask = torch.tensor([1 if node_id_mapping[i] in indices_dict['train_indices'] else 0 for i in range(x.shape[0])], dtype=torch.bool)
valid_mask = torch.tensor([1 if node_id_mapping[i] in indices_dict['valid_indices'] else 0 for i in range(x.shape[0])], dtype=torch.bool)
test_mask = torch.tensor([1 if node_id_mapping[i] in indices_dict['test_indices'] else 0 for i in range(x.shape[0])], dtype=torch.bool)

In [ ]:
data = Data(x=x, y=y, edge_index=edge_index, train_mask=train_mask, valid_mask=valid_mask, test_mask=test_mask)

# GNN Model

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, hidden_channels, output_dim, seed):
        super().__init__()
        torch.cuda.manual_seed(seed)
        # TODO: Complete this function

    def forward(self, x, edge_index):
        # TODO: Complete this function
        pass

# Training

In [ ]:
output_dim = len(subject_mapping)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [ ]:
def evaluate(best_model, rel_mask):
    best_model.eval()
    preds = model(data.x, data.edge_index).argmax(dim=1)
    correct = (preds[rel_mask] == data.y[rel_mask]).sum()
    return round(int(correct) / int(rel_mask.sum()), 3)

In [ ]:
def train():
    # TODO: Complete this function (you can decide what are the input arguments to this function). Make sure you save the model to be used for evaluation as 'best_model.pt'
    pass

# Evaluation

In [ ]:
test_scores = []
for seed in range(1, 4):
    model = GraphSAGE(x.shape[1], output_dim, seed).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    # TODO: Change the call for train if needed
    train()
    best_model = torch.load('best_model.pt')
    curr_seed_score = evaluate(best_model, data.test_mask)
    test_scores.append(curr_seed_score)
    print(f"Seed: {seed}, Test score: {curr_seed_score}")

In [ ]:
print(test_scores)